In [ ]:
# 1. Star
# 2. Folk
# 3. Watching
# 4. Commits
# 5. Issue
# 6. PL 
# 7. Pull Request
# 8. Actions 
# 9. Docs 
# 10. Contributor
# 11. Code

In [2]:
import pandas as pd
import os
import re
import json
from collections import defaultdict
from datetime import datetime, timezone
import pytz

In [ ]:
# the metrics calculated will be saved in '../metrics.csv'

In [3]:
repos_data_path = "/Users/zhangyujin/Desktop/repos-data"
metrics_path = "../metrics.csv"
metrics = pd.read_csv(metrics_path)

In [ ]:
# 01 repo_size

In [6]:
metrics['stars'] = None
metrics['forks'] = None
metrics['watchers'] = None
metrics['open_issues'] = None
metrics['network_count'] = None
metrics['subscribers_count'] = None
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    repo_path = f"{repos_data_path}/{owner}_{repo}"
    if os.path.exists(repo_path):
        repo_info_path = f"{repos_data_path}/{owner}_{repo}/repo.json"
        if os.path.exists(repo_info_path):
            with open(repo_info_path, 'r') as f:
                repo_info = json.load(f)
            metrics.at[index, 'stars'] = f"{repo_info.get('stargazers_count', 0):d}"
            metrics.at[index, 'forks'] = f"{repo_info.get('forks_count', 0):d}"
            metrics.at[index, 'watchers'] = f"{repo_info.get('watchers', 0):d}"
            metrics.at[index, 'open_issues'] = f"{repo_info.get('open_issues', 0):d}"
            metrics.at[index, 'network_count'] = f"{repo_info.get('network_count', 0):d}"
            metrics.at[index, 'subscribers_count'] = f"{repo_info.get('subscribers_count', 0):d}"

In [11]:
metrics.to_csv(metrics_path, index=False)

In [ ]:
# 02 longitudinal(Recent 500 issues/pulls/commits)

In [51]:
metrics['issue_closed_rate'] = None
metrics['issue_time_to_close'] = None
metrics['issue_time_to_response'] = None
metrics['issue_monthly'] = None
metrics['issue_participants'] = None
metrics['issue_per_participants']= None

In [73]:
metrics['issue_commented_rate'] = None

In [4]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        closed_issues_count = 0
        labeled_issues_count = 0
        commented_issues_count = 0
        total_comments = 0
        total_time_to_comment = 0
        total_time_to_close = 0
        monthly_issues_count = defaultdict(int)
        participants = set()
        issues_count = 0
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
            
        for issue in issues[:500]:
            issues_count += 1
#             labeled_issues_count += 1 if issue['labels'] else 0
#             created_at = datetime.datetime.strptime(issue['created_at'], '%Y-%m-%dT%H:%M:%SZ')
#             monthly_issues_count[(created_at.year, created_at.month)] += 1

            if issue['state'] == 'closed':
                closed_issues_count += 1
#                 closed_at = datetime.datetime.strptime(issue['closed_at'], '%Y-%m-%dT%H:%M:%SZ')
#                 total_time_to_close += (closed_at - created_at).total_seconds()

#             comments_path = f"{repos_data_path}/{owner}_{repo}/comments/{issue['number']}.json"
            
#             if os.path.exists(comments_path):
                
#                 with open(comments_path, 'r') as f:
#                     comments = json.load(f)
                    
#                 total_comments += len(comments)
#                 first_comment_time = datetime.datetime.strptime(comments[0]['created_at'], '%Y-%m-%dT%H:%M:%SZ')
#                 time_to_comment = (first_comment_time - created_at).total_seconds()
#                 total_time_to_comment += time_to_comment
#                 commented_issues_count += 1
                
#                 for comment in comments:
#                     participant = comment['user']['login']
#                     participants.add(participant)
        
    metrics.at[index, 'issue_closed_rate'] = f"{(closed_issues_count / issues_count):.2f}" if issues_count > 0 else "N/A"
#     metrics.at[index, 'issue_time_to_close'] = f"{(total_time_to_close / closed_issues_count / (24 * 3600)):.1f}" if closed_issues_count > 0 else "N/A"
#     metrics.at[index, 'issue_time_to_response'] = f"{(total_time_to_comment / commented_issues_count / (24 * 3600)):.1f}" if commented_issues_count > 0 else "N/A"
#     metrics.at[index, 'issue_monthly'] = f"{(sum(monthly_issues_count.values()) / len(monthly_issues_count)):.1f}" if len(monthly_issues_count) > 0 else "N/A"
#     metrics.at[index, 'issue_participants'] = f"{len(participants):d}"
#     metrics.at[index, 'issue_per_participants'] = f"{(len(participants) / issues_count):.1f}" if issues_count > 0 else "N/A"
#     metrics.at[index, 'issue_commented_rate'] = f"{(commented_issues_count / issues_count):.2f}" if issues_count > 0 else "N/A"
    

In [10]:
metrics['labeled_issues'] = None

In [12]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        labeled_issues_count = 0
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
            
        for issue in issues[:500]:
            labeled_issues_count += 1 if issue['labels'] else 0
        
    metrics.at[index, 'labeled_issues'] = f"{labeled_issues_count:d}"

In [53]:
metrics['pulls_closed_rate'] = None
metrics['pulls_merged_rate'] = None
metrics['pulls_time_to_close'] = None
metrics['pulls_time_to_response'] = None
metrics['pulls_monthly'] = None
metrics['pulls_participants'] = None
metrics['pulls_per_participants'] = None

In [54]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    pulls_path = f"{repos_data_path}/{owner}_{repo}/pulls.json"
    
    if os.path.exists(pulls_path):
        pulls_count = 0
        closed_pulls_count = 0
        merged_pulls_count = 0
        commented_pulls_count = 0
        total_comments = 0
        total_time_to_comment = 0
        total_time_to_close = 0
        monthly_pulls_count = defaultdict(int)
        participants = set()
        
        with open(pulls_path, 'r') as f:
            pulls = json.load(f)
            
        if pulls is None:
            continue
            
        for pull in pulls[:500]:
            
            pulls_count += 1
            
            if pull['merged_at']:
                merged_pulls_count += 1
                
            created_at = datetime.datetime.strptime(pull['created_at'], '%Y-%m-%dT%H:%M:%SZ')
            monthly_pulls_count[(created_at.year, created_at.month)] += 1

            if pull['state'] == 'closed':
                closed_pulls_count += 1
                closed_at = datetime.datetime.strptime(pull['closed_at'], '%Y-%m-%dT%H:%M:%SZ')
                total_time_to_close += (closed_at - created_at).total_seconds()

            comments_path = f"{repos_data_path}/{owner}_{repo}/comments/{pull['number']}.json"
            
            if os.path.exists(comments_path):
                
                with open(comments_path, 'r') as f:
                    comments = json.load(f)
                    
                total_comments += len(comments)
                first_comment_time = datetime.datetime.strptime(comments[0]['created_at'], '%Y-%m-%dT%H:%M:%SZ')
                time_to_comment = (first_comment_time - created_at).total_seconds()
                total_time_to_comment += time_to_comment
                commented_pulls_count += 1
                
                for comment in comments:
                    participant = comment['user']['login']
                    participants.add(participant)
        
    metrics.at[index, 'pulls_closed_rate'] = f"{(closed_pulls_count / pulls_count):.1f}" if pulls_count > 0 else "N/A"  
    metrics.at[index, 'pulls_merged_rate'] = f"{(merged_pulls_count / closed_pulls_count):.1f}" if closed_pulls_count > 0 else "N/A"
    metrics.at[index, 'pulls_time_to_close'] = f"{(total_time_to_close / closed_pulls_count / (24 * 3600)):.1f}" if closed_pulls_count > 0 else "N/A"
    metrics.at[index, 'pulls_time_to_response'] = f"{(total_time_to_comment / commented_pulls_count / (24 * 3600)):.1f}" if commented_pulls_count > 0 else "N/A"
    metrics.at[index, 'pulls_monthly'] = f"{(sum(monthly_pulls_count.values()) / len(monthly_pulls_count)):.1f}" if len(monthly_pulls_count) > 0 else "N/A"
    metrics.at[index, 'pulls_participants'] = f"{len(participants):d}"
    metrics.at[index, 'pulls_per_participants'] = f"{(len(participants) / pulls_count):.1f}" if pulls_count > 0 else "N/A"  

In [18]:
metrics['issues_average_comments'] = None
metrics['pulls_average_comments'] = None

In [7]:
metrics['issue_most_recent_time'] = None

In [10]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        total_comments = 0
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
        metrics.at[index, 'issue_most_recent_time'] = f"{issues[0]['created_at']}" if issues else "N/A"

In [21]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    pulls_path = f"{repos_data_path}/{owner}_{repo}/pulls2.json"
    
    if os.path.exists(pulls_path):
        total_comments = 0
        
        with open(pulls_path, 'r') as f:
            pulls = json.load(f)
            
        for pull in pulls[:500]:
            total_comments += pull['comments']
        
    metrics.at[index, 'pulls_average_comments'] = f"{(total_comments / len(pulls)):.1f}" if pulls else "N/A"

In [42]:
metrics['code_review_score'] = None
metrics['maintained_score'] = None

In [43]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    scorecard_path = f"{repos_data_path}/{owner}_{repo}/scorecard.json"
    if os.path.exists(scorecard_path):
        with open(scorecard_path, 'r') as f:
            scorecard = json.load(f)
    if scorecard and scorecard['checks']:
        for check in scorecard['checks']:
            if check['name'] == "Code-Review":
                metrics.at[index, 'code_review_score'] = f"{check['score']}"
            if check['name'] == "Maintained":
                metrics.at[index, 'maintained_score'] = f"{check['score']}"

In [48]:
metrics['vulnerabilities']= None

In [49]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    scorecard_path = f"{repos_data_path}/{owner}_{repo}/scorecard.json"
    if os.path.exists(scorecard_path):
        with open(scorecard_path, 'r') as f:
            scorecard = json.load(f)
    if scorecard and scorecard['checks']:
        for check in scorecard['checks']:
            if check['name'] == "Vulnerabilities":
                metrics.at[index, 'vulnerabilities'] = f"{check['reason'].split()[0]}"

In [57]:
metrics['dependent_repositories'] = None
metrics['dependent_packages'] = None

In [58]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    dependents_path = f"{repos_data_path}/{owner}_{repo}/dependents.json"
    if os.path.exists(dependents_path):
        with open(dependents_path, 'r') as f:
            dependents = json.load(f)
        metrics.at[index, 'dependent_repositories'] = f"{dependents.get('repositories', '0')}"
        metrics.at[index, 'dependent_packages'] = f"{dependents.get('packages', 0)}"

In [70]:
metrics['commits_with_vertification_rate'] = None
metrics['commits_avg_msg_length'] = None
metrics['commits_author_count'] = None
metrics['commits_committer_count'] = None

In [71]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    commits_path = f"{repos_data_path}/{owner}_{repo}/commits.json"
    if os.path.exists(commits_path):
        commits_with_vertification_count = 0
        commits_count = 0
        commits_msg_length = 0
        authors = set()
        committers = set()
        with open(commits_path, 'r') as f:
            commits = json.load(f)
        if commits is None:
            continue
        for commit in commits:
            commits_count += 1
            if commit['author']:
                authors.add(commit['author']['login'])
            if commit['committer']:
                committers.add(commit['committer']['login'])
            commits_msg_length += len(commit['commit']['message'])
            if commit['commit']['verification']['verified'] is True:
                commits_with_vertification_count += 1
        metrics.at[index, 'commits_with_vertification_rate'] = f"{(commits_with_vertification_count / commits_count):.1f}" if commits_count > 0 else "N/A"  
        metrics.at[index, 'commits_avg_msg_length'] = f"{(commits_msg_length / commits_count):.1f}" if commits_count > 0 else "N/A"  
        metrics.at[index, 'commits_author_count'] = f"{len(authors):d}"
        metrics.at[index, 'commits_committer_count'] = f"{len(committers):d}"

In [76]:
metrics['readme_headers_num'] = None

In [80]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    readme_path = f"{repos_data_path}/{owner}_{repo}/readme.md"
    if os.path.exists(readme_path):
        heading_levels = {
            'h1': re.compile(r'^(#\s.*)', re.MULTILINE),
            'h2': re.compile(r'^(##\s.*)', re.MULTILINE),
            'h3': re.compile(r'^(###\s.*)', re.MULTILINE)
        }
        counts = 0
        with open(readme_path, 'r', encoding='utf-8') as file:
            for line in file:
                for level, regex in heading_levels.items():
                    matches = regex.findall(line)
                    counts += len(matches)
        metrics.at[index, 'readme_headers_num'] = counts 

In [3]:
metrics['comments_avg_length'] = None

In [4]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        comments_count = 0
        comments_length = 0
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
            
        for issue in issues[:500]:
            comments_path = f"{repos_data_path}/{owner}_{repo}/comments/{issue['number']}.json"
            
            if os.path.exists(comments_path):
                with open(comments_path, 'r') as f:
                    comments = json.load(f)
                for comment in comments:
                    comments_length += len(comment['body'])
                    comments_count += 1
        
    metrics.at[index, 'comments_avg_length'] = f"{(comments_length / comments_count):.2f}" if comments_count > 0 else "N/A"

In [6]:
metrics['pulls_last_90days'] = None

In [16]:
for index, row in metrics.iterrows():
    target_date_aware = datetime(2023, 12, 1, tzinfo=pytz.utc)
    owner = row['owner']
    repo = row['repo']
    pulls_count = 0
    pulls_path = f"{repos_data_path}/{owner}_{repo}/pulls.json"
    if os.path.exists(pulls_path):
        with open(pulls_path, 'r') as f:
            pulls = json.load(f)
        if pulls is None:
            continue
        for pull in pulls[:500]:
            date_str = pull['created_at']
            date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            date_obj_aware = date_obj.replace(tzinfo=pytz.utc)
            if date_obj_aware > target_date_aware:
                pulls_count += 1
    metrics.at[index, 'pulls_last_90days'] = f"{pulls_count:d}" if pulls_count > 0 else "N/A"
    

In [19]:
metrics['commmits_last_90days'] = None

In [20]:
for index, row in metrics.iterrows():
    target_date_aware = datetime(2023, 12, 1, tzinfo=pytz.utc)
    owner = row['owner']
    repo = row['repo']
    commits_count = 0
    commits_path = f"{repos_data_path}/{owner}_{repo}/commits.json"
    if os.path.exists(commits_path):
        with open(commits_path, 'r') as f:
            commits = json.load(f)
        if commits is None:
            continue
        for commit in commits[:500]:
            date_str = commit['commit']['committer']['date']
            date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            date_obj_aware = date_obj.replace(tzinfo=pytz.utc)
            if date_obj_aware > target_date_aware:
                commits_count += 1
    metrics.at[index, 'commmits_last_90days'] = f"{commits_count:d}" if commits_count > 0 else "N/A"
    

In [4]:
metrics['commit_most_recent_time'] = None

In [5]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    commits_path = f"{repos_data_path}/{owner}_{repo}/commits.json"
    if os.path.exists(commits_path):
        with open(commits_path, 'r') as f:
            commits = json.load(f)
        metrics.at[index, 'commit_most_recent_time'] = f"{commits[0]['commit']['author']['date']}" if commits else "N/A"

In [7]:
metrics['health_percentage'] = None

In [10]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    profile_path = f"{repos_data_path}/{owner}_{repo}/profile.json"
    if os.path.exists(profile_path):
        with open(profile_path, 'r') as f:
            profile = json.load(f)
        metrics.at[index, 'health_percentage'] = f"{profile.get('health_percentage')}" if profile else "N/A"

In [12]:
metrics['projects_owned_by_maintainer'] = None
metrics['age_of_other_projects'] = None

In [16]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    maintainer_path = f"{repos_data_path}/{owner}_{repo}/maintainer.json"
    today = datetime.strptime("2024-03-31T23:59:59Z", '%Y-%m-%dT%H:%M:%SZ')
    age = 0
    if os.path.exists(maintainer_path):
        with open(maintainer_path, 'r') as f:
            maintainer = json.load(f)
        for repo in maintainer:
            created_at = datetime.strptime(repo['created_at'], '%Y-%m-%dT%H:%M:%SZ')
            age += (today - created_at).total_seconds()
    metrics.at[index, 'projects_owned_by_maintainer'] = f"{len(maintainer)}" if maintainer else "N/A"
    metrics.at[index, 'age_of_other_projects'] = f"{age / len(maintainer) / (3600*24):.1f}" if maintainer else "N/A"

2019-05-25T08:45:17Z
2014-06-03T16:58:49Z
2013-07-31T22:56:59Z
2013-08-03T00:20:12Z
2015-07-01T00:59:41Z
2021-04-22T08:26:05Z
2014-06-14T19:33:02Z
2015-06-30T16:32:35Z
2015-09-26T04:21:17Z
2013-12-07T23:28:59Z
2015-06-30T17:05:28Z
2017-09-20T04:22:59Z
2013-08-03T00:01:50Z
2015-06-30T23:09:31Z
2015-08-19T18:53:44Z
2019-09-12T00:18:34Z
2010-03-09T21:16:26Z
2020-07-13T19:51:44Z
2023-10-09T22:32:52Z
2020-02-21T04:23:10Z
2022-06-30T17:47:07Z
2019-06-12T21:50:32Z
2016-04-06T23:29:35Z
2016-04-05T17:56:50Z
2014-09-24T18:27:03Z
2015-08-27T12:13:11Z
2016-02-14T13:48:55Z
2014-05-02T00:30:39Z
2014-05-05T20:47:58Z
2014-08-12T18:05:02Z
2014-08-05T23:15:01Z
2014-08-06T17:30:51Z
2015-05-05T20:34:34Z
2014-05-22T18:50:03Z
2016-10-28T23:33:37Z
2019-07-08T19:31:49Z
2014-12-05T22:00:00Z
2022-10-19T10:22:12Z
2016-08-02T21:14:39Z
2019-11-05T15:14:16Z
2017-09-15T14:29:57Z
2016-02-18T18:55:09Z
2020-04-05T23:10:56Z
2019-06-26T02:07:17Z
2014-09-18T20:03:16Z
2019-05-25T08:45:17Z
2014-06-03T16:58:49Z
2013-07-31T22

2018-03-08T19:39:52Z
2021-01-27T18:39:16Z
2015-05-20T22:56:51Z
2021-11-10T11:33:40Z
2016-04-04T12:54:35Z
2016-04-04T13:14:17Z
2017-12-07T02:42:30Z
2017-12-07T12:18:20Z
2016-09-13T04:49:42Z
2017-02-06T18:12:10Z
2016-06-26T18:13:26Z
2017-08-14T14:19:24Z
2016-06-26T15:14:55Z
2021-05-26T08:12:20Z
2014-08-04T15:18:28Z
2017-02-28T14:51:46Z
2016-07-08T12:58:58Z
2015-12-30T18:26:11Z
2017-03-13T03:57:14Z
2015-01-07T13:57:09Z
2015-07-01T11:56:56Z
2015-07-04T11:11:08Z
2016-05-04T22:53:26Z
2015-08-14T15:23:20Z
2016-04-27T09:26:10Z
2015-04-09T07:39:56Z
2016-03-23T14:12:10Z
2019-08-08T22:37:20Z
2015-05-18T17:36:45Z
2015-07-18T14:59:49Z
2018-03-08T19:39:52Z
2021-01-27T18:39:16Z
2015-05-20T22:56:51Z
2021-11-10T11:33:40Z
2016-04-04T12:54:35Z
2016-04-04T13:14:17Z
2017-12-07T02:42:30Z
2017-12-07T12:18:20Z
2016-09-13T04:49:42Z
2017-02-06T18:12:10Z
2016-06-26T18:13:26Z
2017-08-14T14:19:24Z
2016-06-26T15:14:55Z
2021-05-26T08:12:20Z
2014-08-04T15:18:28Z
2017-02-28T14:51:46Z
2016-07-08T12:58:58Z
2015-12-30T18

2021-08-04T22:35:42Z
2020-12-10T19:52:01Z
2021-10-04T12:52:35Z
2016-02-13T00:39:52Z
2019-10-23T19:04:50Z
2021-06-11T05:27:16Z
2017-11-09T11:07:20Z
2017-02-05T08:40:37Z
2015-08-27T00:58:22Z
2015-08-27T00:59:12Z
2017-04-20T19:03:17Z
2015-06-23T03:01:27Z
2015-06-23T20:41:35Z
2015-12-05T17:17:22Z
2014-07-25T02:32:30Z
2014-08-22T22:59:48Z
2020-12-27T12:23:18Z
2016-10-04T03:24:37Z
2015-08-06T22:44:04Z
2016-11-01T04:50:16Z
2015-10-23T09:15:23Z
2015-11-19T13:38:05Z
2016-10-07T11:07:05Z
2018-10-09T11:11:08Z
2017-07-30T05:45:41Z
2019-01-04T05:26:13Z
2020-08-21T06:34:51Z
2014-09-15T23:57:30Z
2017-04-14T04:43:21Z
2021-05-09T19:12:31Z
2018-03-08T19:39:52Z
2021-01-27T18:39:16Z
2015-05-20T22:56:51Z
2021-11-10T11:33:40Z
2016-04-04T12:54:35Z
2016-04-04T13:14:17Z
2017-12-07T02:42:30Z
2017-12-07T12:18:20Z
2016-09-13T04:49:42Z
2017-02-06T18:12:10Z
2016-06-26T18:13:26Z
2017-08-14T14:19:24Z
2016-06-26T15:14:55Z
2021-05-26T08:12:20Z
2014-08-04T15:18:28Z
2017-02-28T14:51:46Z
2016-07-08T12:58:58Z
2015-12-30T18

2019-09-17T07:02:25Z
2014-10-03T01:53:04Z
2019-03-22T21:39:27Z
2014-11-18T06:41:58Z
2019-04-07T17:53:48Z
2011-08-04T22:05:21Z
2014-01-02T23:27:02Z
2019-11-02T19:10:18Z
2014-06-02T06:34:28Z
2013-01-12T20:07:28Z
2014-08-05T07:46:55Z
2017-11-01T20:38:34Z
2019-01-30T19:13:44Z
2014-09-11T22:52:50Z
2012-12-10T22:56:54Z
2014-02-26T17:42:20Z
2014-03-02T18:31:34Z
2019-04-26T21:25:32Z
2014-02-26T17:41:52Z
2020-07-19T20:25:38Z
2014-10-13T06:26:32Z
2017-09-12T20:17:42Z
2014-01-02T23:29:16Z
2013-02-08T00:26:36Z
2013-01-13T07:36:52Z
2015-08-24T18:18:37Z
2015-04-13T15:43:13Z
2019-06-26T05:12:56Z
2019-06-13T18:58:47Z
2014-10-28T01:55:59Z
2020-02-29T10:28:43Z
2010-05-07T15:50:21Z
2023-04-13T17:32:24Z
2022-02-18T21:36:58Z
2022-07-18T21:34:53Z
2022-11-19T14:49:22Z
2024-01-11T23:33:08Z
2023-11-02T21:52:32Z
2022-09-15T18:55:37Z
2022-02-24T22:50:59Z
2015-07-18T23:31:47Z
2022-03-04T00:24:53Z
2022-11-01T21:16:32Z
2023-11-21T00:44:04Z
2014-05-16T22:22:51Z
2020-07-03T00:36:49Z
2022-11-09T20:26:03Z
2020-10-26T19

2022-01-12T14:03:23Z
2016-12-05T11:44:48Z
2016-03-31T09:45:46Z
2016-10-26T15:50:08Z
2021-11-19T10:54:07Z
2016-04-21T12:17:42Z
2016-02-16T14:14:29Z
2021-04-08T13:22:52Z
2016-12-01T22:10:30Z
2017-02-17T21:25:23Z
2017-05-01T20:27:05Z
2016-11-27T22:16:56Z
2016-06-10T16:01:41Z
2017-07-06T09:34:23Z
2017-03-23T15:46:08Z
2016-10-20T07:56:41Z
2016-04-20T19:37:01Z
2016-07-21T17:59:50Z
2018-10-22T16:47:31Z
2018-06-18T17:16:57Z
2016-10-21T09:23:49Z
2016-10-25T01:50:37Z
2020-05-15T09:41:30Z
2016-07-28T15:52:59Z
2018-03-30T23:41:58Z
2017-03-01T00:53:42Z
2016-05-17T15:15:39Z
2018-02-21T00:34:55Z
2016-06-22T23:58:00Z
2016-12-09T22:56:44Z
2022-01-17T06:25:04Z
2017-03-07T17:11:14Z
2021-10-04T05:41:44Z
2011-11-18T18:03:54Z
2011-11-18T18:04:11Z
2013-04-21T18:29:32Z
2014-02-18T20:28:41Z
2013-01-18T23:18:25Z
2018-07-19T21:38:38Z
2015-12-18T21:17:41Z
2015-11-18T21:01:19Z
2015-12-01T06:50:57Z
2021-06-22T05:17:20Z
2010-03-11T18:24:48Z
2014-03-07T16:44:00Z
2021-04-15T10:07:15Z
2024-03-20T15:01:33Z
2024-03-22T16

2020-05-21T00:06:29Z
2024-02-29T15:43:31Z
2014-12-10T21:08:37Z
2014-11-29T05:37:02Z
2014-01-08T08:02:40Z
2014-01-06T08:25:03Z
2014-01-01T23:38:37Z
2013-08-10T16:12:39Z
2013-04-09T20:19:12Z
2014-01-16T02:59:48Z
2014-02-15T16:22:04Z
2014-02-23T10:52:57Z
2013-01-31T03:27:53Z
2014-03-05T06:45:25Z
2016-02-25T01:59:47Z
2013-03-14T09:54:43Z
2014-03-03T07:58:51Z
2023-11-11T17:26:37Z
2009-06-26T18:56:01Z
2011-04-07T23:03:00Z
2011-02-21T19:27:17Z
2014-06-12T18:58:35Z
2012-06-06T03:08:03Z
2014-07-03T00:41:11Z
2014-09-04T19:56:18Z
2013-11-20T22:24:21Z
2014-03-03T08:06:33Z
2014-04-19T14:42:12Z
2014-02-08T19:19:14Z
2014-01-30T18:16:04Z
2015-07-08T16:12:31Z
2015-01-14T18:15:17Z
2017-07-22T00:41:54Z
2014-04-23T07:34:03Z
2015-07-17T09:52:05Z
2017-09-23T16:26:52Z
2015-07-08T10:08:06Z
2015-07-08T10:15:19Z
2018-03-27T14:31:48Z
2017-07-21T14:12:06Z
2014-08-18T22:22:46Z
2015-04-24T16:33:13Z
2014-04-25T07:59:13Z
2014-02-07T09:10:24Z
2013-08-27T18:25:42Z
2014-11-21T10:53:20Z
2015-05-22T18:13:09Z
2015-07-06T13

2019-06-15T08:44:40Z
2018-11-03T16:22:21Z
2018-12-01T10:04:14Z
2016-08-06T02:03:53Z
2015-11-29T14:39:41Z
2017-12-07T17:41:23Z
2015-05-18T23:59:17Z
2020-05-17T14:13:44Z
2014-07-28T19:35:14Z
2015-12-27T14:09:11Z
2017-04-28T15:41:00Z
2017-07-20T20:08:59Z
2016-01-14T20:08:56Z
2015-04-15T11:41:55Z
2015-06-24T04:54:57Z
2015-08-06T12:32:33Z
2020-10-22T17:08:48Z
2017-07-14T13:52:30Z
2017-01-24T17:03:56Z
2020-03-29T13:46:41Z
2017-07-20T20:16:58Z
2016-02-16T12:28:46Z
2020-10-04T09:42:34Z
2018-06-03T10:44:28Z
2015-03-16T11:50:35Z
2016-01-17T19:03:30Z
2015-09-08T22:18:19Z
2015-06-21T17:24:17Z
2019-02-28T17:04:52Z
2015-07-12T17:19:40Z
2019-09-13T22:12:05Z
2016-01-21T10:25:33Z
2020-07-07T14:02:30Z
2017-11-08T16:34:17Z
2015-05-11T11:44:01Z
2017-03-22T16:15:12Z
2019-09-12T15:43:45Z
2020-07-02T09:39:27Z
2021-07-16T08:22:15Z
2020-09-03T10:26:08Z
2021-09-08T15:27:28Z
2017-03-23T02:19:26Z
2015-09-03T21:03:56Z
2020-08-17T16:08:51Z
2021-05-25T15:57:12Z
2016-11-22T15:30:33Z
2015-12-08T03:06:05Z
2015-12-07T10

TypeError: string indices must be integers, not 'str'